# **Overview**

This script processes medical image segmentation data in `.nii.gz` format. The main steps include:
1. **Reading NIfTI Files**: The script reads `.nii.gz` files representing segmentation data.
2. **Processing the Data**: It processes the segmentation arrays, curates the largest mask for specific labels (1 to 4), and combines them into a final segmentation array.
3. **Saving the Processed Data**: The processed segmentation arrays are saved as new NIfTI files.

---

## **Key Steps**

### **1. Saving Numpy Arrays to NIfTI Files**
- **Function** `save_array_to_nii`: Converts a numpy array to a NIfTI image and saves it with the same spacing, direction, and origin as the template NIfTI.

### **2. File Path Matching**
- A directory is searched for `.nii.gz` files, and paths to the files are collected.

### **3. Data Processing**
- For each segmentation file:
  - The array is split by labels (1 through 4).
  - Each label array is processed to retain only the largest connected mask for that label.
  - The individual label arrays are combined into a final segmentation array.

### **4. Saving Results**
- The processed segmentation array is saved as a new NIfTI file using the `save_array_to_nii` function.

---

## **Requirements**
- Python packages: `SimpleITK`, `numpy`, `os`, `shutil`, and custom helper functions.


In [7]:
# CELL 1: Importing Required Libraries and Defining Helper Function
import SimpleITK as sitk
import numpy as np
from skimage import measure
# Function to save a numpy array as a NIfTI file with metadata from a template image
def save_array_to_nii(mask_array, nii_template_path, nii_save_path):

    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()
#     print(output_spacing ,output_direction,output_origin)

    nrrd_output = sitk.GetImageFromArray(mask_array)
    nrrd_output.SetSpacing(output_spacing)
    nrrd_output.SetDirection(output_direction)
    nrrd_output.SetOrigin(output_origin)

    nrrdWriter = sitk.ImageFileWriter()
    nrrdWriter.SetFileName(nii_save_path)
    nrrdWriter.SetUseCompression(True)
    nrrdWriter.Execute(nrrd_output)
    print(nii_save_path ,'saved')

In [8]:
# CELL 2: Collecting Image Paths
import os

# Directory containing segmentation files
data_dir = 'nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/'

# Collect and sort paths of all .nii.gz files in the directory
image_paths = sorted([os.path.join(r, file) for r, d, f in os.walk(data_dir) for file in f if '.nii.gz' in file])

# Print the number of images found
print(f"Number of images found: {len(image_paths)}")


Number of images found: 2


In [9]:
# CELL 3: Processing and Saving the Curated Segmentation
from test_scripts.image_postprocessing_test import get_biggst_mask_array

# Directory to save the processed images
targ_dir = 'nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/'

# Iterate through each segmentation image path
for idx, img_path in enumerate(image_paths):
    print(f"Processing {idx+1}/{len(image_paths)}: {img_path}")

    # Read the segmentation image
    seg_array = sitk.GetArrayFromImage(sitk.ReadImage(img_path))

    # Process each label (1 through 4), isolating and keeping the largest connected component
    processed_arrays = []
    for label in range(1, 5):
        label_array = np.copy(seg_array)
        label_array[label_array != label] = 0  # Keep only pixels of the current label
        label_array[label_array != 0] = 1  # Set non-zero pixels to 1
        processed_arrays.append(get_biggst_mask_array(label_array) * label)  # Keep the largest component

    # Combine the processed arrays into the final segmentation
    final_seg_array = sum(processed_arrays)

    # Define the save path for the processed image
    target_path = os.path.join(targ_dir, os.path.basename(img_path).replace('.nii.gz', '_largest_curated.nii.gz'))

    # Save the final segmentation array to a NIfTI file
    save_array_to_nii(final_seg_array, img_path, target_path)
    print(f"Processed and saved: {target_path}\n")


Processing 1/2: nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/TUM01_fat_img_img_L2-L5_nnunet_to_seg.nii.gz
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/TUM01_fat_img_img_L2-L5_nnunet_to_seg_largest_curated.nii.gz saved
Processed and saved: nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/TUM01_fat_img_img_L2-L5_nnunet_to_seg_largest_curated.nii.gz

Processing 2/2: nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw/TUM01_water_img_img_L2-L5_nnunet_to_seg.nii.gz
nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/TUM01_water_img_img_L2-L5_nnunet_to_seg_largest_curated.nii.gz saved
Processed and saved: nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/TUM01_water_img_img_L2-L5_nnunet_to_seg_largest_curated.nii.gz

